# Import

In [1]:
import os
import sys

import time
import math
import json
import glob
from datetime import datetime

import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize, least_squares
import matplotlib.pyplot as plt

import cv2
import torch
from torch.nn.functional import mse_loss
from pytorch_kinematics.transforms.rotation_conversions import matrix_to_axis_angle, axis_angle_to_matrix

import pytorch_kinematics as pk
from pytransform3d.urdf import UrdfTransformManager
from urdf_parser_py.urdf import URDF

# Helper Functions

In [2]:
def load_json(filename: str):
    # Construct the absolute path by joining the current directory and relative path
    absolute_path = os.path.join(os.getcwd(), '..', filename)
    # Load the JSON data
    with open(absolute_path, 'r') as file:
        json_data = json.load(file)    
    return json_data

def xml_to_str(robot):
    # To XML string with filtering
    temp_str = robot.to_xml_string()
    words = temp_str.split()
    words[5] = '>'
    urdf_str = ' '.join(words)
    return urdf_str

def active_matrix_from_angle(basis, angle):
    c = torch.cos(angle)  # Shape: Nx1
    s = torch.sin(angle)  # Shape: Nx1
    
    if basis == 0:
        R = torch.zeros((angle.size(0), 3, 3), dtype=angle.dtype, device=angle.device)
        R[:, 0, 0] = 1.0
        R[:, 1, 1] = c.squeeze()  # Shape: N
        R[:, 1, 2] = -s.squeeze()  # Shape: N
        R[:, 2, 1] = s.squeeze()    # Shape: N
        R[:, 2, 2] = c.squeeze()    # Shape: N
    elif basis == 1:
        R = torch.zeros((angle.size(0), 3, 3), dtype=angle.dtype, device=angle.device)
        R[:, 0, 0] = c.squeeze()    # Shape: N
        R[:, 0, 2] = s.squeeze()    # Shape: N
        R[:, 1, 1] = 1.0
        R[:, 2, 0] = -s.squeeze()    # Shape: N
        R[:, 2, 2] = c.squeeze()    # Shape: N
    elif basis == 2:
        R = torch.zeros((angle.size(0), 3, 3), dtype=angle.dtype, device=angle.device)
        R[:, 0, 0] = c.squeeze()    # Shape: N
        R[:, 0, 1] = -s.squeeze()   # Shape: N
        R[:, 1, 0] = s.squeeze()    # Shape: N
        R[:, 1, 1] = c.squeeze()    # Shape: N
        R[:, 2, 2] = 1.0
    else:
        raise ValueError("Basis must be in [0, 1, 2]")
    
    return R

def active_matrix_from_extrinsic_euler_xyz(alpha, beta, gamma):
    # Calculate the rotation matrices
    R_alpha = active_matrix_from_angle(0, alpha)  # Rotation around x-axis
    R_beta = active_matrix_from_angle(1, beta)    # Rotation around y-axis
    R_gamma = active_matrix_from_angle(2, gamma)   # Rotation around z-axis

    # Combine the matrices using the @ operator
    R = R_gamma @ (R_beta @ R_alpha)
    return R

def get_homogeneous_matrix(roll,pitch,yaw,x,y,z):
    rot_mat = active_matrix_from_extrinsic_euler_xyz(roll,pitch,yaw)
    H = torch.zeros((rot_mat.size(0), 4, 4), dtype=dtype, device=device)
    H[:, :3, :3] = rot_mat
    H[:, 0, 3] = x.squeeze()
    H[:, 1, 3] = y.squeeze()
    H[:, 2, 3] = z.squeeze()
    H[:, 3, 3] = 1.0
    return H

def get_pred_rvec_tvec(V, 
                       cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, 
                       l_rvec, l_tvec, T_home):
    
    # Convert to Tensor
    cmd_lnt = torch.Tensor(cmd_lnt).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lnp = torch.Tensor(cmd_lnp).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_unt = torch.Tensor(cmd_unt).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_et = torch.Tensor(cmd_et).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lep = torch.Tensor(cmd_lep).reshape(-1,1).to(dtype=dtype, device=device)

    l_rvec_t = torch.Tensor(l_rvec).to(dtype=dtype, device=device)
    l_tvec_t = torch.Tensor(l_tvec).to(dtype=dtype, device=device)

    ### Joint-to-Motor Polynomial Models
    x = torch.Tensor(cmd_lnt).reshape(-1,1).to(dtype=dtype, device=device)
    lnt_roll = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[0]],[V[1]],[V[2]],[V[3]]]).to(dtype=dtype, device=device))
    lnt_pitch = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[4]],[V[5]],[V[6]],[V[7]]]).to(dtype=dtype, device=device))
    lnt_yaw = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[8]],[V[9]],[V[10]],[V[11]]]).to(dtype=dtype, device=device))
    lnt_x = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[12]],[V[13]],[V[14]],[V[15]]]).to(dtype=dtype, device=device))
    lnt_y = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[16]],[V[17]],[V[18]],[V[19]]]).to(dtype=dtype, device=device))
    lnt_z = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[20]],[V[21]],[V[22]],[V[23]]]).to(dtype=dtype, device=device))

    x = torch.Tensor(cmd_lnp).reshape(-1,1).to(dtype=dtype, device=device)
    lnp_roll = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[24]],[V[25]],[V[26]],[V[27]]]).to(dtype=dtype, device=device))
    lnp_pitch = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[28]],[V[29]],[V[30]],[V[31]]]).to(dtype=dtype, device=device))
    lnp_yaw = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[32]],[V[33]],[V[34]],[V[35]]]).to(dtype=dtype, device=device))
    lnp_x = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[36]],[V[37]],[V[38]],[V[39]]]).to(dtype=dtype, device=device))
    lnp_y = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[40]],[V[41]],[V[42]],[V[43]]]).to(dtype=dtype, device=device))
    lnp_z = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[44]],[V[45]],[V[46]],[V[47]]]).to(dtype=dtype, device=device))

    x = torch.Tensor(cmd_unt).reshape(-1,1).to(dtype=dtype, device=device)
    unt_roll = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[48]],[V[49]],[V[50]],[V[51]]]).to(dtype=dtype, device=device))
    unt_pitch = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[52]],[V[53]],[V[54]],[V[55]]]).to(dtype=dtype, device=device))
    unt_yaw = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[56]],[V[57]],[V[58]],[V[59]]]).to(dtype=dtype, device=device))
    unt_x = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[60]],[V[61]],[V[62]],[V[63]]]).to(dtype=dtype, device=device))
    unt_y = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[64]],[V[65]],[V[66]],[V[67]]]).to(dtype=dtype, device=device))
    unt_z = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[68]],[V[69]],[V[70]],[V[71]]]).to(dtype=dtype, device=device))

    x = torch.Tensor(cmd_et).reshape(-1,1).to(dtype=dtype, device=device)
    et_roll = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[72]],[V[73]],[V[74]],[V[75]]]).to(dtype=dtype, device=device))
    et_pitch = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[76]],[V[77]],[V[78]],[V[79]]]).to(dtype=dtype, device=device))
    et_yaw = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[80]],[V[81]],[V[82]],[V[83]]]).to(dtype=dtype, device=device))
    et_x = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[84]],[V[85]],[V[86]],[V[87]]]).to(dtype=dtype, device=device))
    et_y = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[88]],[V[89]],[V[90]],[V[91]]]).to(dtype=dtype, device=device))
    et_z = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[92]],[V[93]],[V[94]],[V[95]]]).to(dtype=dtype, device=device))

    x = torch.Tensor(cmd_lep).reshape(-1,1).to(dtype=dtype, device=device)
    lep_roll = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[96]],[V[97]],[V[98]],[V[99]]]).to(dtype=dtype, device=device))
    lep_pitch = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[100]],[V[101]],[V[102]],[V[103]]]).to(dtype=dtype, device=device))
    lep_yaw = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[104]],[V[105]],[V[106]],[V[107]]]).to(dtype=dtype, device=device))
    lep_x = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[108]],[V[109]],[V[110]],[V[111]]]).to(dtype=dtype, device=device))
    lep_y = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[112]],[V[113]],[V[114]],[V[115]]]).to(dtype=dtype, device=device))
    lep_z = (torch.cat((torch.ones(x.shape[0], 1), x, x.pow(2), x.pow(3)), dim=1) @ torch.Tensor([[V[116]],[V[117]],[V[118]],[V[119]]]).to(dtype=dtype, device=device))
    
    # Home Homogeneous
    T_temp = torch.Tensor(T_home).to(dtype=dtype, device=device)
    T_0 = torch.stack([T_temp] * cmd_lnt.size(0))

    # Homogeneous Matrices
    T_lnt = get_homogeneous_matrix(lnt_roll,lnt_pitch,lnt_yaw,lnt_x,lnt_y,lnt_z)
    T_lnp = get_homogeneous_matrix(lnp_roll,lnp_pitch,lnp_yaw,lnp_x,lnp_y,lnp_z)
    T_unt = get_homogeneous_matrix(unt_roll,unt_pitch,unt_yaw,unt_x,unt_y,unt_z)
    T_et = get_homogeneous_matrix(et_roll,et_pitch,et_yaw,et_x,et_y,et_z)
    T_lep = get_homogeneous_matrix(lep_roll,lep_pitch,lep_yaw,lep_x,lep_y,lep_z)

    # Final Matrix
    T_final = T_0 @ T_lnt @ T_lnp @ T_unt @ T_et @ T_lep
    T_final

    # Predicted
    pred_l_rvec = matrix_to_axis_angle(T_final[:,:3,:3])
    pred_l_tvec = T_final[:,:3,3]
    
    # Return Output
    return pred_l_rvec, pred_l_tvec


def get_total_loss(data_dir, data_csv, fit_result):
    # Initialization
    data_fn = '241003_075m_grace_dataset.csv'
    data_dir = 'thesis'
    files_path = os.path.join(os.getcwd(),"..",'data', data_dir, data_fn)
    df = pd.read_csv(files_path)

    # Filtered df
    home_df = df[(df['cmd_theta_lower_neck_tilt']==0) 
                 & (df['cmd_theta_lower_neck_pan']==0) 
                 & (df['cmd_theta_upper_neck_tilt']==0)
                 & (df['cmd_theta_left_eye_pan']==0)
                 & (df['cmd_theta_right_eye_pan']==0)
                 & (df['cmd_theta_eyes_tilt']==0)]
    home_df = home_df.reset_index(drop=True)

    # Rvec and Tvec
    home_l_rvec = np.array([home_df['l_rvec_0'],home_df['l_rvec_1'],home_df['l_rvec_2']])
    home_l_tvec = np.array([home_df['l_tvec_0'],home_df['l_tvec_1'],home_df['l_tvec_2']])
    temp_rmat,_ = cv2.Rodrigues(home_l_rvec)
    T_home = np.eye(4)
    T_home[:3,:3] = temp_rmat.squeeze()
    T_home[:3,3] = home_l_tvec.flatten() 
    T_home_inv = np.linalg.inv(T_home)
    
    # Load Dataset CSV
    csv_file = os.path.join(os.getcwd(),"..","data", data_dir, data_csv)
    temp_df = pd.read_csv(csv_file)
    
    # Convert DF Degrees Column to Radians
    data_df = temp_df.copy()
    data_df['cmd_theta_lower_neck_pan'] = np.radians(data_df['cmd_theta_lower_neck_pan'].values)
    data_df['cmd_theta_lower_neck_tilt'] = np.radians(data_df['cmd_theta_lower_neck_tilt'].values)
    data_df['cmd_theta_upper_neck_tilt'] = np.radians(data_df['cmd_theta_upper_neck_tilt'].values)
    data_df['cmd_theta_left_eye_pan'] = np.radians(data_df['cmd_theta_left_eye_pan'].values)
    data_df['cmd_theta_right_eye_pan'] = np.radians(data_df['cmd_theta_right_eye_pan'].values)
    data_df['cmd_theta_eyes_tilt'] = np.radians(data_df['cmd_theta_eyes_tilt'].values)
    
    # Load CSV Optimization Results
    csv_path = os.path.join(os.getcwd(),"..","results",data_dir,fit_result)
    csv_df = pd.read_csv(csv_path, index_col=0)
    var_dict = dict(zip(csv_df.index,csv_df['learned'].values))
    
    # Decision Variables 
    var_list = list(var_dict.values())

    # Variable Names
    var_names_list = list(var_dict.keys())

    # Mapping
    idx2var = dict(zip(list(range(len(var_names_list))),var_names_list))
    var2idx = dict(zip(var_names_list, list(range(len(var_names_list)))))
    
    # Input
    V = np.array(var_list)
    z_c_r = data_df['z_c_r'].to_numpy()
    cmd_lnt = data_df['cmd_theta_lower_neck_tilt'].to_numpy()
    cmd_lnp = data_df['cmd_theta_lower_neck_pan'].to_numpy()
    cmd_unt = data_df['cmd_theta_upper_neck_tilt'].to_numpy()
    cmd_et = data_df['cmd_theta_eyes_tilt'].to_numpy()
    cmd_lep = data_df['cmd_theta_left_eye_pan'].to_numpy()
    cmd_rep = data_df['cmd_theta_right_eye_pan'].to_numpy()

    l_rvec = np.concatenate([data_df['l_rvec_0'].values.reshape(-1,1),
                             data_df['l_rvec_1'].values.reshape(-1,1),
                             data_df['l_rvec_2'].values.reshape(-1,1)], axis=1)
    l_tvec = np.concatenate([data_df['l_tvec_0'].values.reshape(-1,1),
                             data_df['l_tvec_1'].values.reshape(-1,1),
                             data_df['l_tvec_2'].values.reshape(-1,1)], axis=1)
    r_rvec = np.concatenate([data_df['r_rvec_0'].values.reshape(-1,1),
                             data_df['r_rvec_1'].values.reshape(-1,1),
                             data_df['r_rvec_2'].values.reshape(-1,1)], axis=1)
    r_tvec = np.concatenate([data_df['r_tvec_0'].values.reshape(-1,1),
                             data_df['r_tvec_1'].values.reshape(-1,1),
                             data_df['r_tvec_2'].values.reshape(-1,1)], axis=1)
    
    # Forward Kinematics
    out_l_rvec, out_l_tvec = get_pred_rvec_tvec(V, 
                                               cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, 
                                               l_rvec, l_tvec, T_home)
    # True Value
    out_l_rvec = out_l_rvec.to(dtype=dtype, device=device)
    out_l_tvec = out_l_tvec.to(dtype=dtype, device=device)
    
    # True Value
    l_rvec_t = torch.Tensor(l_rvec).to(dtype=dtype, device=device)
    l_tvec_t = torch.Tensor(l_tvec).to(dtype=dtype, device=device)
    
    # Results
    results_idx = ['r(rad)','p(rad)', 'y(rad)', 'x(cm)','y(cm)', 'z(cm)',]
    results_df = pd.DataFrame({'Left Eye RMSE': [
                                math.sqrt(mse_loss(out_l_rvec[:,0], l_rvec_t[:,0]).cpu().item()),
                                math.sqrt(mse_loss(out_l_rvec[:,1], l_rvec_t[:,1]).cpu().item()),
                                math.sqrt(mse_loss(out_l_rvec[:,2], l_rvec_t[:,2]).cpu().item()),
                                math.sqrt(mse_loss(100*out_l_tvec[:,0], 100*l_tvec_t[:,0]).cpu().item()),
                                math.sqrt(mse_loss(100*out_l_tvec[:,1], 100*l_tvec_t[:,1]).cpu().item()),
                                math.sqrt(mse_loss(100*out_l_tvec[:,2], 100*l_tvec_t[:,2]).cpu().item()),
                                ],
                              'Left Eye MSE': [
                                mse_loss(out_l_rvec[:,0], l_rvec_t[:,0]).cpu().item(),
                                mse_loss(out_l_rvec[:,1], l_rvec_t[:,1]).cpu().item(),
                                mse_loss(out_l_rvec[:,2], l_rvec_t[:,2]).cpu().item(),
                                mse_loss(out_l_tvec[:,0], l_tvec_t[:,0]).cpu().item(),
                                mse_loss(out_l_tvec[:,1], l_tvec_t[:,1]).cpu().item(),
                                mse_loss(out_l_tvec[:,2], l_tvec_t[:,2]).cpu().item(),
                                ],
                              })
    results_df.index = results_idx
    
    return results_df

# Initialization

In [3]:
# Explicitly telling to use GPU
torch.set_default_device('cuda')
torch.set_default_dtype(d=torch.float64)
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64

# Single Data

In [14]:
single_df = get_total_loss(data_dir="thesis", data_csv="241003_075m_grace_dataset.csv", 
                           fit_result="../20241015_073936_420250_rpy_075m_grace_results.csv")
single_df

,Left Eye RMSE,Left Eye MSE
r(rad),0.026226,0.000688
p(rad),0.021186,0.000449
y(rad),0.079967,0.006395
x(cm),1.262815,0.000159
y(cm),0.488883,0.000024
z(cm),1.077992,0.000116


In [15]:
single_df = get_total_loss(data_dir="thesis", data_csv="241005_075m_test_grace_dataset_1000.csv", 
                           fit_result="../20241015_073936_420250_rpy_075m_grace_results.csv")
single_df

,Left Eye RMSE,Left Eye MSE
r(rad),0.027564,0.000760
p(rad),0.016428,0.000270
y(rad),0.060215,0.003626
x(cm),1.040233,0.000108
y(cm),0.429916,0.000018
z(cm),0.939040,0.000088


# Multiple Optimization (Model 1)

In [ ]:
data_list = ['241010_best_075m_grace_dataset_50.csv',
             '241010_best_075m_grace_dataset_100.csv',
             '241010_best_075m_grace_dataset_250.csv',
             '241010_best_075m_grace_dataset_500.csv',
             '241010_best_075m_grace_dataset_750.csv',
             '241010_best_075m_grace_dataset_1000.csv']

In [ ]:
test_list = len(data_list)*['241005_075m_test_grace_dataset_1000.csv']

In [ ]:
fit_list = ['20241010_025410_831165_50_075m_grace_frozen_results.csv',
            '20241010_025429_164830_100_075m_grace_frozen_results.csv',
            '20241010_025449_498409_250_075m_grace_frozen_results.csv',
            '20241010_025509_909690_500_075m_grace_frozen_results.csv',
            '20241010_025528_619187_750_075m_grace_frozen_results.csv',
            '20241010_025543_764379_1000_075m_grace_frozen_results.csv']  

## Train Set

In [ ]:
model1_train_df_list = []
model1_train_mse_loss_list = []
model1_train_rad_rmse_loss_list = []
model1_train_cm_rmse_loss_list = []
for i in range(len(data_list)):
    temp_df = get_total_loss_frozen(data_dir="thesis", data_csv=data_list[i], fit_result=fit_list[i])
    model1_train_df_list.append(temp_df.copy())
    
    total_mse_loss = np.sum(temp_df['Left Eye MSE']) + np.sum(temp_df['Right Eye MSE'])
    model1_train_mse_loss_list.append(total_mse_loss)
    
    rad_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][:3]) + np.sum(temp_df['Right Eye RMSE'][:3]))/6
    model1_train_rad_rmse_loss_list.append(rad_rmse_loss)
    
    cm_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][3:]) + np.sum(temp_df['Right Eye RMSE'][3:]))/6
    model1_train_cm_rmse_loss_list.append(cm_rmse_loss)

## Test Set

In [ ]:
model1_test_df_list = []
model1_test_mse_loss_list = []
model1_test_rad_rmse_loss_list = []
model1_test_cm_rmse_loss_list = []
for i in range(len(data_list)):
    temp_df = get_total_loss_frozen(data_dir="thesis", data_csv=test_list[i], fit_result=fit_list[i])
    model1_test_df_list.append(temp_df.copy())
    
    total_mse_loss = np.sum(temp_df['Left Eye MSE']) + np.sum(temp_df['Right Eye MSE'])
    model1_test_mse_loss_list.append(total_mse_loss)
    
    rad_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][:3]) + np.sum(temp_df['Right Eye RMSE'][:3]))/6
    model1_test_rad_rmse_loss_list.append(rad_rmse_loss)
    
    cm_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][3:]) + np.sum(temp_df['Right Eye RMSE'][3:]))/6
    model1_test_cm_rmse_loss_list.append(cm_rmse_loss)

# Multiple Optimization (Model 2)

In [ ]:
data_list = ['241010_best_075m_grace_dataset_50.csv',
             '241010_best_075m_grace_dataset_100.csv',
             '241010_best_075m_grace_dataset_250.csv',
             '241010_best_075m_grace_dataset_500.csv',
             '241010_best_075m_grace_dataset_750.csv',
             '241010_best_075m_grace_dataset_1000.csv']

In [ ]:
test_list = len(data_list)*['241005_075m_test_grace_dataset_1000.csv']

In [ ]:
fit_list = ['20241010_024324_007672_50_075m_grace_results.csv',
            '20241010_024410_523423_100_075m_grace_results.csv',
            '20241010_024521_902733_250_075m_grace_results.csv',
            '20241010_024613_791140_500_075m_grace_results.csv',
            '20241010_024710_840319_750_075m_grace_results.csv',
            '20241010_024803_056417_1000_075m_grace_results.csv']  

## Train Set

In [ ]:
model2_train_df_list = []
model2_train_mse_loss_list = []
model2_train_rad_rmse_loss_list = []
model2_train_cm_rmse_loss_list = []
for i in range(len(data_list)):
    temp_df = get_total_loss(data_dir="thesis", data_csv=data_list[i], fit_result=fit_list[i])
    model2_train_df_list.append(temp_df.copy())
    
    total_mse_loss = np.sum(temp_df['Left Eye MSE']) + np.sum(temp_df['Right Eye MSE'])
    model2_train_mse_loss_list.append(total_mse_loss)
    
    rad_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][:3]) + np.sum(temp_df['Right Eye RMSE'][:3]))/6
    model2_train_rad_rmse_loss_list.append(rad_rmse_loss)
    
    cm_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][3:]) + np.sum(temp_df['Right Eye RMSE'][3:]))/6
    model2_train_cm_rmse_loss_list.append(cm_rmse_loss)

## Test Set

In [ ]:
model2_test_df_list = []
model2_test_mse_loss_list = []
model2_test_rad_rmse_loss_list = []
model2_test_cm_rmse_loss_list = []
for i in range(len(data_list)):
    temp_df = get_total_loss(data_dir="thesis", data_csv=test_list[i], fit_result=fit_list[i])
    model2_test_df_list.append(temp_df.copy())
    
    total_mse_loss = np.sum(temp_df['Left Eye MSE']) + np.sum(temp_df['Right Eye MSE'])
    model2_test_mse_loss_list.append(total_mse_loss)
    
    rad_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][:3]) + np.sum(temp_df['Right Eye RMSE'][:3]))/6
    model2_test_rad_rmse_loss_list.append(rad_rmse_loss)
    
    cm_rmse_loss = (np.sum(temp_df['Left Eye RMSE'][3:]) + np.sum(temp_df['Right Eye RMSE'][3:]))/6
    model2_test_cm_rmse_loss_list.append(cm_rmse_loss)

# Total Plot

In [ ]:
plt.plot(model1_test_mse_loss_list,'-o')
plt.plot(model2_test_mse_loss_list,'-o')
plt.legend(['Model 1 Test','Model 2 Test'])
plt.xticks(ticks=list(range(len(model2_test_mse_loss_list))), labels=['50','100','250','500','750','1000'])
plt.xlabel('Dataset N')
plt.ylabel('Total MSE Loss')
plt.grid()

In [ ]:
plt.plot(model1_test_rad_rmse_loss_list,'-o')
plt.plot(model2_test_rad_rmse_loss_list,'-o')
plt.legend(['Model 1 Test','Model 2 Test'])
plt.xticks(ticks=list(range(len(model2_test_mse_loss_list))), labels=['50','100','250','500','750','1000'])
plt.xlabel('Dataset N')
plt.ylabel('Mean RMSE Loss (rad)')
plt.grid()

In [ ]:
plt.plot(model1_test_cm_rmse_loss_list,'-o')
plt.plot(model2_test_cm_rmse_loss_list,'-o')
plt.legend(['Model 1 Test','Model 2 Test'])
plt.xticks(ticks=list(range(len(model2_test_mse_loss_list))), labels=['50','100','250','500','750','1000'])
plt.xlabel('Dataset N')
plt.ylabel('Mean RMSE Loss (cm)')
plt.grid()